In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import requests, json

In [2]:
cd data/

D:\Documents\Tugas\Anaconda\Skripsi_Recommender\data_50u_35i\data


In [3]:
names = ['user_id', 'item_id', 'rating']
df = pd.read_csv('reviews_50u_35i_ready.csv', sep=',', names=names)

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

In [4]:
print str(n_users) + ' users'
print str(n_items) + ' items'
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print 'Sparsity: {:4.2f}%'.format(sparsity)

8625 users
25877 items
Sparsity: 0.41%


In [5]:
from ExplicitMF import ExplicitMF

In [6]:
best_sgd_model = ExplicitMF(ratings, n_factors=20, learning='sgd', \
                            item_fact_reg=0.1, user_fact_reg=0.1, \
                            user_bias_reg=0.1, item_bias_reg=0.1)
best_sgd_model.train(200, learning_rate=0.001)

In [7]:
from IPython.display import HTML

def display_top_k_books(similarity, idx_to_book, book_idx, k=10):
    book_indices = np.argsort(similarity[book_idx,:])[::-1]
    book_idx_list = ''
    k_ctr = 0
    # Start i at 1 to not grab the input book
    i = 0
    while k_ctr < 10:
        book = idx_to_book[book_indices[i]]
        if book != '':
            book_idx_list += "<h4>- %s<h4/>"\
                      % book
            k_ctr += 1
        i += 1
    display(HTML(book_idx_list))

def compare_recs(sgd_similarity,\
                 idx_to_book, book_idx, k=10):

    # Display SGD Recs
    display(HTML('<font size=5>'+'SGD Recs'+'</font>'))
    display_top_k_books(sgd_similarity,\
                          idx_to_book, book_idx)

In [8]:
sgd_sim = best_sgd_model.cosine_similarity()

In [9]:
import io
# Load in book data
idx_to_book = {}

with io.open('reviews_50u_35i_info_items.csv', mode='r', encoding='utf-8-sig') as f:
    for line in f.readlines():
        info = line.split(',')        
        if len(info[1]) <= 10:
            for asin_char in xrange(len(info[1]), 11):
                info[1] = '0' + info[1]
        
        idx_to_book[int(info[0])-1] = info[0] + ' : ' + info[1]    

In [11]:
idx = 20000 
compare_recs(sgd_sim, idx_to_book, idx)

In [ ]:
import cPickle as pickle

with open("python_best_sgd_model.pkl", "wb") as file_handler:
    pickle.dump(best_sgd_model, file_handler)

In [20]:
with open("python_best_sgd_model.pkl", "rb") as file_handler:
    loaded_pickle = pickle.load(file_handler)
    
loaded_pickle

EOFError: 

In [ ]:
users = ratings.shape[0]
books = ratings.shape[1]
books_mean = np.zeros((1, books))
books_norm = np.zeros((users, books))

R = np.zeros((users,books))
for us in range(users):
    for mo in range(books):
        if ratings[us,mo] >= 1.0:
            R[us,mo] = 1
        else:
            R[us,mo] = 0

for i in range(books):
    idx = np.where(R[:,i] == 1)[0]
    books_mean[0][i] = ratings[idx,i].mean()
    books_norm[idx,i] = ratings[idx,i] - books_mean[0][i]

In [ ]:
best_sgd_model2 = ExplicitMF(books_norm, n_factors=20, learning='sgd', \
                            item_fact_reg=0.1, user_fact_reg=0.1, \
                            user_bias_reg=0.1, item_bias_reg=0.1)
best_sgd_model2.train(200, learning_rate=0.001)

In [ ]:
with open("python_best_sgd_model2.pkl", "wb") as file_handler:
    pickle.dump(best_sgd_model, file_handler)

In [ ]:
my_ratings = np.zeros((1682, 1))

my_ratings[587] = 5